# 包 `tqdm`

`tqdm` 是Python中一个进度条库，可以为任何可迭代对象包装一个智能进度条。

`tqdm` 本质上是一个类，其构造函数包含众多参数，包括：
- `iterable`: 可迭代对象（如列表、生成器等）
- `desc`: 字符串，进度条前的描述文本
- `total`: 预期的迭代总数，默认为None
- `leave`: 布尔值，是否保留进度条的最终状态（默认True）
- `ncols`: 整数，进度条的显示宽度
- `mininterval`: 浮点数，最小更新间隔（秒），默认0.1秒
- `maxinterval`: 浮点数，最大更新间隔（秒），默认10秒
- `ascii`: 布尔值，是否使用ASCII字符显示
- `disable`: 布尔值，是否禁用进度条显示
- `unit`: 字符串，进度单位，默认为'it'（iterations）

In [1]:
from tqdm import tqdm
import time

for i in tqdm(range(100), 
              desc="处理中", 
              unit="文件", 
              unit_scale=True):
    time.sleep(0.01)

处理中: 100%|██████████| 100/100 [00:01<00:00, 63.9文件/s] 


# 包 `ray`

Ray是一个开源的统一分布式计算框架。

## 环境初始化与集群连接

### 本地单机初始化
```python
import ray
import numpy as np

# 初始化Ray运行时环境
# 如果不指定参数，Ray会自动检测本机资源
ray.init()

# 或者指定具体资源配置
ray.init(
    num_cpus=8,           # 指定CPU核心数
    num_gpus=2,           # 指定GPU数量（如果有）
    object_store_memory=2000000000,  # 对象存储内存大小（字节）
    ignore_reinit_error=True         # 允许重复初始化
)
```
### 连接到远程Ray集群

```python
import ray

# 连接到已存在的Ray集群
ray.init(address='ray://head-node-ip:10001')
```

## 定义远程函数和Actor类
### 定义无状态的远程函数（适用于并行任务）
```python
@ray.remote
def simple_remote_function(input_data, processing_params=None):
    """
    无状态的远程函数模板
    Args:
        input_data: 输入数据
        processing_params: 处理参数
    Returns:
        processed_result: 处理结果
    """
    
    # 实际的处理逻辑

    return # 返回值

# 指定资源需求的远程函数
@ray.remote(num_cpus=2, num_gpus=0.5, memory=1000*1024*1024)
def resource_intensive_function(large_data):
    """需要特定资源的函数"""
    # CPU或GPU密集型计算
    processed_data = complex_computation(large_data)
    return processed_data
```
### 定义有状态的Actor类（适用于需要维护状态的任务）
```python
@ray.remote
class StatefulProcessor:
    """
    有状态的远程Actor类模板
    """
    def __init__(self, initial_config=None):
        """初始化Actor状态"""
        self.state = {}
        self.config = initial_config or {}
        self.processed_count = 0
    
    def process_batch(self, data_batch):
        """批量处理数据"""
        results = []
        for item in data_batch:
            # 处理单个数据项
            processed_item = self._process_single_item(item)
            results.append(processed_item)
            self.processed_count += 1
        
        return {
            'batch_results': results,
            'total_processed': self.processed_count
        }
    
    def _process_single_item(self, item):
        """处理单个数据项的私有方法"""
        # 根据当前状态和配置处理数据
        return item * self.config.get('multiplier', 1)
    
    def get_status(self):
        """获取当前状态"""
        return {
            'processed_count': self.processed_count,
            'state_size': len(self.state),
            'config': self.config
        }
    
    def update_config(self, new_config):
        """更新配置"""
        self.config.update(new_config)
        return f"配置已更新: {self.config}"

# 指定资源需求的Actor
@ray.remote(num_cpus=4, memory=2000*1024*1024)
class ResourceIntensiveActor:
    """需要大量资源的Actor"""
    def __init__(self):
        # 初始化大型数据结构或模型
        self.large_state = np.zeros((10000, 10000))  # 大型状态
    
    def heavy_computation(self, input_data):
        # 计算密集型操作
        return np.dot(self.large_state, input_data)
```

## 数据准备与存储优化
### 将大型数据对象存储到Ray对象存储
```python
# 准备大型数据集
large_dataset = np.random.rand(100000, 100)  

# 将大型数据存储到分布式对象存储（零拷贝共享）
large_data_ref = ray.put(large_dataset)
```
### 数据分片策略
```python
def create_data_chunks(data, chunk_size):
    """
    将大型数据集分割成小块用于并行处理
    Args:
        data: 原始数据
        chunk_size: 每块的大小
    Returns:
        List[Any]: 数据块列表
    """
    chunks = []
    for i in range(0, len(data), chunk_size):
        chunk = data[i:i + chunk_size]
        chunks.append(chunk)
    return chunks

# 将数据分割成块
original_data = list(range(10000))  # 模拟原始数据
data_chunks = create_data_chunks(original_data, chunk_size=1000)

# 将数据块存储到对象存储
chunk_refs = []
for i, chunk in enumerate(data_chunks):
    chunk_ref = ray.put(chunk)
    chunk_refs.append(chunk_ref)
```

## 任务提交与并行执行
### 无状态任务的并行执行
```python
# 提交多个并行任务
task_futures = []
for i, chunk_ref in enumerate(chunk_refs):
    # 异步提交任务，立即返回Future对象
    future = simple_remote_function.remote(chunk_ref, processing_params=2)
    task_futures.append(future)

# 方式2: 批量提交（更简洁）
batch_futures = [
    simple_remote_function.remote(chunk_ref, processing_params=3) 
    for chunk_ref in chunk_refs
]
```
### 有状态Actor的使用
```python
# 创建多个Actor实例
num_actors = 3
actors = []

for i in range(num_actors):
    actor_config = {'multiplier': i + 1, 'actor_id': i}
    actor = StatefulProcessor.remote(initial_config=actor_config)
    actors.append(actor)

# 使用Actor处理数据
actor_futures = []

for i, actor in enumerate(actors):
    # 为每个Actor分配数据块
    assigned_chunks = chunk_refs[i::num_actors]  # 轮流分配
    
    for chunk_ref in assigned_chunks:
        future = actor.process_batch.remote([chunk_ref])
        actor_futures.append(future)
```
### 混合执行模式
```python
# 同时使用函数和Actor进行不同类型的计算
# 轻量级预处理任务（使用远程函数）
preprocessing_futures = [
    simple_remote_function.remote(chunk_ref, processing_params=0.5)
    for chunk_ref in chunk_refs[:5]  # 处理前5个块
]

# 重量级状态计算任务（使用Actor）
stateful_futures = []
for i, actor in enumerate(actors):
    future = actor.process_batch.remote(chunk_refs[5 + i:5 + i + 1])
    stateful_futures.append(future)
```

## 结果收集与进度监控
### 等待任务完成并收集结果
```python
import time

# 方式1: 等待所有任务完成
start_time = time.time()
# 阻塞等待所有任务完成
all_results = ray.get(task_futures)
execution_time = time.time() - start_time

# 方式2: 逐步收集已完成的任务
remaining_futures = batch_futures.copy()
completed_results = []

while remaining_futures:
    # 等待至少一个任务完成
    ready_futures, remaining_futures = ray.wait(
        remaining_futures, 
        num_returns=1,      # 每次等待1个任务完成
        timeout=10.0        # 最多等待10秒
    )
    
    if ready_futures:
        # 收集已完成的结果
        batch_results = ray.get(ready_futures)
        completed_results.extend(batch_results)
        
        progress = len(completed_results) / len(batch_futures) * 100
        print(f"进度: {progress:.1f}% ({len(completed_results)}/{len(batch_futures)})")
    else:
        print("等待超时，继续等待...")

print(f"逐步收集完成，共收到 {len(completed_results)} 个结果")
```
### Actor状态监控
```python
# 监控Actor的状态
print("监控Actor状态...")

actor_status_futures = [actor.get_status.remote() for actor in actors]
actor_statuses = ray.get(actor_status_futures)

for i, status in enumerate(actor_statuses):
    print(f"Actor {i} 状态: {status}")

# 获取Actor的计算结果
print("收集Actor计算结果...")
if actor_futures:
    actor_results = ray.get(actor_futures)
    print(f"从Actor收到 {len(actor_results)} 个计算结果")
```
### 实时进度监控
```python
def monitor_progress(futures, check_interval=2.0):
    """
    实时监控任务进度
    Args:
        futures: 任务Future列表
        check_interval: 检查间隔（秒）
    """
    total_tasks = len(futures)
    completed_count = 0
    
    print(f"开始监控 {total_tasks} 个任务的执行进度...")
    
    while completed_count < total_tasks:
        # 检查已完成的任务数量
        ready, not_ready = ray.wait(futures, timeout=0)
        completed_count = len(ready)
        
        progress_percent = (completed_count / total_tasks) * 100
        remaining_tasks = total_tasks - completed_count
        
        print(f"进度: {progress_percent:.1f}% | 已完成: {completed_count} | 剩余: {remaining_tasks}")
        
        if completed_count < total_tasks:
            time.sleep(check_interval)
    
    print("所有任务执行完成！")
    return ray.get(futures)

# 使用进度监控
if task_futures:
    monitored_results = monitor_progress(task_futures[:5], check_interval=1.0)
```

## 结果聚合与后处理
### 结果数据整合
```python
def aggregate_results(results):
    """
    聚合多个任务的计算结果
    Args:
        results: 任务结果列表
    Returns:
        Dict: 聚合后的结果统计
    """
    if not results:
        return {'error': '没有结果需要聚合'}
    
    # 统计聚合
    total_processed = len(results)
    successful_results = [r for r in results if r.get('status') == 'completed']
    failed_results = [r for r in results if r.get('status') != 'completed']
    
    # 数值聚合（如果适用）
    if successful_results and 'result' in successful_results[0]:
        try:
            numeric_results = [r['result'] for r in successful_results if isinstance(r['result'], (int, float))]
            if numeric_results:
                aggregated_stats = {
                    'sum': sum(numeric_results),
                    'average': sum(numeric_results) / len(numeric_results),
                    'min': min(numeric_results),
                    'max': max(numeric_results),
                    'count': len(numeric_results)
                }
            else:
                aggregated_stats = {'message': '没有可聚合的数值结果'}
        except Exception as e:
            aggregated_stats = {'error': f'聚合计算失败: {str(e)}'}
    else:
        aggregated_stats = {'message': '结果不包含可聚合的数值'}
    
    return {
        'total_tasks': total_processed,
        'successful_tasks': len(successful_results),
        'failed_tasks': len(failed_results),
        'success_rate': len(successful_results) / total_processed * 100,
        'aggregated_stats': aggregated_stats,
        'sample_results': successful_results[:3]  # 展示前3个样本结果
    }

# 聚合所有结果
if all_results:
    final_summary = aggregate_results(all_results)
    print("=== 最终结果汇总 ===")
    print(f"任务总数: {final_summary['total_tasks']}")
    print(f"成功任务: {final_summary['successful_tasks']}")
    print(f"失败任务: {final_summary['failed_tasks']}")
    print(f"成功率: {final_summary['success_rate']:.2f}%")
    print(f"聚合统计: {final_summary['aggregated_stats']}")
```
### 性能分析
```python
def analyze_performance():
    """分析Ray集群性能"""
    print("=== 性能分析 ===")
    
    # 集群资源使用情况
    cluster_resources = ray.cluster_resources()
    available_resources = ray.available_resources()
    
    print("集群总资源:")
    for resource, total in cluster_resources.items():
        available = available_resources.get(resource, 0)
        used = total - available
        usage_percent = (used / total) * 100 if total > 0 else 0
        print(f"  {resource}: {used:.1f}/{total:.1f} ({usage_percent:.1f}% 使用)")
    
    # 对象存储使用情况
    print("\n对象存储使用情况:")
    memory_stats = ray.object_store_memory_usage()
    print(f"  使用内存: {memory_stats}")
    
    # 节点状态
    nodes = ray.nodes()
    print(f"\n集群节点数: {len(nodes)}")
    alive_nodes = [node for node in nodes if node['Alive']]
    print(f"活跃节点数: {len(alive_nodes)}")

# 执行性能分析
analyze_performance()
```

## 资源清理与环境恢复
### 清理Actor资源
```python
# 显式销毁Actor实例，释放资源
print("清理Actor资源...")
for i, actor in enumerate(actors):
    ray.kill(actor)  # 主动销毁Actor
    print(f"Actor {i} 已销毁")

# 清理大型对象引用
print("清理对象存储...")
del large_data_ref, chunk_refs  # 删除Python引用

# 手动触发垃圾回收（可选）
import gc
gc.collect()
```
### 关闭Ray运行时
```python
# 查看最终资源使用情况
print("=== 清理前资源状态 ===")
analyze_performance()

# 关闭Ray集群连接
print("关闭Ray运行时...")
ray.shutdown()

print("Ray运行时已关闭")
print(f"Ray是否仍在运行: {ray.is_initialized()}")
```